In [25]:

import os
import sys
import pandas as pd
import numpy as np
import create_dicts


code_path = os.getcwd()
data_path = "C:/Users/XHK/Desktop/thesis_code/events_analysis/data/raw/kaggle"

event_type1, event_type2, side, shot_place, shot_outcome, location, \
    bodypart, assist_method, situation = create_dicts.get_dictionaries()

events = pd.read_csv('/'.join([data_path, "events_w_weighted_french.csv"]))
ginf = pd.read_csv('/'.join([data_path, "ginf.csv"]))

events_columns = ['id_odsp', 'id_event', 'sort_order', 'time', 'text', 'event_type',
                  'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2',
                  'player_in', 'player_out', 'shot_place', 'shot_outcome', 'is_goal',
                  'location', 'bodypart', 'assist_method', 'situation', 'fast_break']

ginf_columns = ['id_odsp', 'link_odsp', 'adv_stats', 'date', 'league', 'season',
                'country', 'ht', 'at', 'fthg', 'ftag', 'odd_h', 'odd_d', 'odd_a',
                'odd_over', 'odd_under', 'odd_bts', 'odd_bts_n'] 


Getting dictionaries to analyse events.csv


Some code to determine which league is best to start analysing: 

the variable ginf is a DataFrame created from a .csv which stores general info about each match in the 'events' dataset

We want detailed match-data, which means the match must have adv_stats == True

In [26]:
ginf.groupby(['adv_stats', 'league']).count()


id_odsp  link_odsp  date  season  country    ht    at  fthg  \
adv_stats league                                                                
False     D1           49         49    49      49       49    49    49    49   
          E0          820        820   820     820      820   820   820   820   
          F1           19         19    19      19       19    19    19    19   
          I1           18         18    18      18       18    18    18    18   
          SP1          24         24    24      24       24    24    24    24   
True      D1         1641       1641  1641    1641     1641  1641  1641  1641   
          E0         1300       1300  1300    1300     1300  1300  1300  1300   
          F1         2088       2088  2088    2088     2088  2088  2088  2088   
          I1         2088       2088  2088    2088     2088  2088  2088  2088   
          SP1        2065       2065  2065    2065     2065  2065  2065  2065   

                  ftag  odd_h  odd_d  odd_a  odd_over  odd_under  odd_bts  \
adv_stats league                                                            
False     D1        49     49     49     49         0          0        0   
          E0       820    820    820    820         0          0        0   
          F1        19     19     19     19         0          0        0   
          I1        18     18     18     18         0          0        0   
          SP1       24     24     24     24         0          0        0   
True      D1      1641   1641   1641   1641       153        153      153   
          E0      1300   1300   1300   1300       220        220      220   
          F1      2088   2088   2088   2088       208        208      208   
          I1      2088   2088   2088   2088       207        207      207   
          SP1     2065   2065   2065   2065       189        189      189   

                  odd_bts_n  
adv_stats league             
False     D1              0  
          E0              0  
          F1              0  
          I1              0  
          SP1             0  
True      D1            153  
          E0            220  
          F1            208  
          I1            207  
          SP1           189

France and Italy have the most matches with available events-data in the dataset. 
We pick the French league arbitrarily, moving forward.

first we want to have a column for home-goal-attempts and away-goal-attempts and add it to ginf. 
attempt has an event_type of 1

In [27]:
matches_no_stats_fr = ginf[(ginf.league == 'F1') & (ginf.adv_stats == False)]

matches_fr = ginf[(ginf.adv_stats == True) & (ginf.league == 'F1')]
events_fr = events.loc[events.id_odsp.isin(matches_fr.id_odsp)]



In [28]:

#matches_fr['home_attempts'] = np.nan
#matches_fr['away_attempts'] = np.nan

h_att = events_fr[(events_fr.event_type == 1) & (
        events_fr.side == 1)].groupby('id_odsp').count()

a_att = events_fr[(events_fr.event_type == 1) & (
        events_fr.side == 2)].groupby('id_odsp').count()



In [29]:
matches_fr.index = matches_fr.id_odsp
matches_fr = pd.concat([matches_fr,h_att.event_type],   axis = 1)
matches_fr = matches_fr.rename({'event_type':'home_attempts' }, axis='columns')
matches_fr = pd.concat([matches_fr,a_att.event_type],   axis = 1)
matches_fr = matches_fr.rename({'event_type':'away_attempts'}, axis='columns')

matches_fr.head()


,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n,home_attempts,away_attempts
CzPV312a/,CzPV312a/,/soccer/france/ligue-1-2011-2012/paris-sg-lori...,True,2011-08-06,F1,2012,france,Paris Saint-Germain,Lorient,0,1,1.55,4.50,9.40,NaN,NaN,NaN,NaN,12.0,9.0
GUOdmtII/,GUOdmtII/,/soccer/france/ligue-1-2011-2012/caen-valencie...,True,2011-08-06,F1,2012,france,Caen,Valenciennes,1,0,2.50,3.40,3.45,NaN,NaN,NaN,NaN,11.0,12.0
M7PhlM2C/,M7PhlM2C/,/soccer/france/ligue-1-2011-2012/brest-evian-t...,True,2011-08-06,F1,2012,france,Brest,Evian Thonon Gaillard,2,2,2.29,3.25,3.85,NaN,NaN,NaN,NaN,23.0,9.0
QuWqjrYa/,QuWqjrYa/,/soccer/france/ligue-1-2011-2012/ac-ajaccio-to...,True,2011-08-06,F1,2012,france,AC Ajaccio,Toulouse,0,2,2.80,3.10,3.05,NaN,NaN,NaN,NaN,18.0,13.0
UBZQ4smg/,UBZQ4smg/,/soccer/france/ligue-1-2011-2012/nice-lyon-UBZ...,True,2011-08-06,F1,2012,france,Nice,Lyon,1,3,4.50,3.55,2.00,NaN,NaN,NaN,NaN,12.0,22.0


We have added the amount of attempts on goal by the home-team and by the away-team. 
There are around 12 NaN's in those columns. We will deal with them by using the average amount of attempts of the corresponding teams:

We proceed by getting the amount of red card events:
In the events.csv, 'location'

First we define a helper function:  

In [30]:
def get_events(events, event_type_nr, type=1 ):
    event_type = 'event_type'
    if type != 1:
        event_type= 'event_type2'
    h_events = events[(events[event_type] == event_type_nr) & (
        events.side == 1)].groupby('id_odsp').count()


    a_events = events[(events[event_type] == event_type_nr) & (
        events.side == 2)].groupby('id_odsp').count()

    return h_events, a_events

def concat_and_rename(df_matches, home_vec, away_vec, name, type = 1):
    event_type = 'event_type'
    if type != 1:
        event_type = 'event_type2'

    home_name = 'home_'+name
    away_name = 'away_'+name 
    df_matches.index = df_matches.id_odsp

    df_matches = pd.concat([df_matches, home_vec[event_type]],   axis=1)
    df_matches = df_matches.rename({event_type: home_name}, axis='columns')
    df_matches = pd.concat([df_matches, away_vec[event_type]],   axis=1)
    df_matches = df_matches.rename({event_type: away_name}, axis='columns')

    # changing nans to zeroes:
    df_matches.loc[df_matches[home_name].isna(), home_name] = 0
    df_matches.loc[df_matches[away_name].isna(), away_name] = 0
    return df_matches

home_redcard,away_redcard = get_events(events_fr, 6)
matches_fr = concat_and_rename(matches_fr, home_redcard, away_redcard, name = 'redcard')


changing all the NaN's to 0 red_card values: 

Adding the 'conceded penalty' feature:       penalty_conceded has eventnumber 11

In [31]:
home_penalty_conceded, away_penalty_conceded = get_events(events_fr, 11)
matches_fr = concat_and_rename(    matches_fr, home_redcard, away_redcard, name='penalty_conceded')



adding columns for a home_sentoff and away_sentoff features

In [32]:
home_sent, away_sent = get_events(events_fr, 14, type=2)
matches_fr = concat_and_rename(    matches_fr, home_sent, away_sent, name='sentoff')


adding corner_conceded features for both teams.   Corners conceded is lower for away team???

In [33]:
home_corner, away_corner = get_events(events_fr, 2, type = 1)
matches_fr = concat_and_rename(
    matches_fr, home_sent, away_sent, name='corner_conceded')


fouls

In [34]:
home_fouls, away_fouls = get_events(events_fr, 3, type=1)
matches_fr = concat_and_rename(    matches_fr, home_sent, away_sent, name='fouls')


[] Use shot-placement, location and isGoal to determine a weighting for attempts
    [use logistic-regression]
[] Make a weighted-attempts feature

In [40]:
#all the French attempt-events that did not end in own-goals 
french_attempt_events = events[(events.id_odsp.isin(matches_fr.id_odsp)) & (events.event_type ==1) & (events.event_type2 !=15)]
french_attempt_events.groupby('id_odsp side'.split()).sum()

Unnamed: 0  sort_order  time  event_type  event_type2  \
id_odsp   side                                                          
00OX4xFp/ 1        5979171         381   326           9         84.0   
          2        5979393         603   535           9         84.0   
00QH2XdM/ 1        7715211         651   579          14         96.0   
          2        6061986         546   483          11         84.0   
00nmICd9/ 1       10588129        1215  1029          19        168.0   
...                    ...         ...   ...         ...          ...   
zwg8Qk7J/ 2        9117059         959   867          15        132.0   
zwhjOjUB/ 1        1261091         531   658          16        144.0   
          2        1024644         439   526          13        156.0   
zwyvizbA/ 1        8223996         591   614          15        120.0   
          2        6031107         610   642          11        120.0   

                shot_place  shot_outcome  is_goal  location  bodypart  \
id_odsp   side                                                          
00OX4xFp/ 1           72.0          16.0        0      92.0      16.0   
          2           37.0          21.0        0      93.0      17.0   
00QH2XdM/ 1           64.0          28.0        3     125.0      28.0   
          2           43.0          19.0        2     105.0      23.0   
00nmICd9/ 1           93.0          37.0        0     234.0      33.0   
...                    ...           ...      ...       ...       ...   
zwg8Qk7J/ 2           90.0          31.0        1     196.0      21.0   
zwhjOjUB/ 1           93.0          31.0        0     187.0      19.0   
          2           77.0          21.0        1     134.0      20.0   
zwyvizbA/ 1           99.0          32.0        0     123.0      24.0   
          2           56.0          14.0        4      85.0      19.0   

                assist_method  situation  fast_break  french_attempt_weight  
id_odsp   side                                                               
00OX4xFp/ 1                15       11.0           0               1.417156  
          2                14       13.0           0               2.149014  
00QH2XdM/ 1                12       19.0           0               1.810257  
          2                11       12.0           0               1.321323  
00nmICd9/ 1                20       22.0           0               1.735353  
...                       ...        ...         ...                    ...  
zwg8Qk7J/ 2                15       16.0           0               0.499977  
zwhjOjUB/ 1                16       19.0           0               1.430716  
          2                23       16.0           0               2.038438  
zwyvizbA/ 1                15       26.0           0               2.285588  
          2                20       11.0           2               2.251902  

[4151 rows x 14 columns]